In [1]:
import pandas as pd

df = pd.read_json('crawler/output.json')

In [2]:
df.head()

,url,title,content
0,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"NvidiaSearchInput.mount({""apiUrl"": ""https://ap..."
1,https://docs.nvidia.com/cuda/cuda-toolkit-rele...,CUDA 12.5 Update 1 Release Notes,1. CUDA 12.5 Update 1 Release Notes \r\n 1.1. ...
2,https://docs.nvidia.com/cuda/cuda-installation...,CUDA Installation Guide for Linux,1. Introduction \n 1.1. System Requirements \n...
3,https://docs.nvidia.com/cuda/cuda-features-arc...,CUDA Features Archive,1. CUDA 11.6 Features \n \n 1.1. Compiler \n 1...
4,https://docs.nvidia.com/cuda/cuda-installation...,CUDA Installation Guide for Microsoft Windows,1. Introduction \n 1.1. System Requirements \n...


In [3]:
df["complete_texts"] = df.apply(lambda x: str(x["title"])+"\n"+str(x["content"]), axis=1)

df.head()

,url,title,content,complete_texts
0,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"NvidiaSearchInput.mount({""apiUrl"": ""https://ap...",CUDA Toolkit Documentation 12.5\nNvidiaSearchI...
1,https://docs.nvidia.com/cuda/cuda-toolkit-rele...,CUDA 12.5 Update 1 Release Notes,1. CUDA 12.5 Update 1 Release Notes \r\n 1.1. ...,CUDA 12.5 Update 1 Release Notes\n1. CUDA 12.5...
2,https://docs.nvidia.com/cuda/cuda-installation...,CUDA Installation Guide for Linux,1. Introduction \n 1.1. System Requirements \n...,CUDA Installation Guide for Linux\n1. Introduc...
3,https://docs.nvidia.com/cuda/cuda-features-arc...,CUDA Features Archive,1. CUDA 11.6 Features \n \n 1.1. Compiler \n 1...,CUDA Features Archive\n1. CUDA 11.6 Features \...
4,https://docs.nvidia.com/cuda/cuda-installation...,CUDA Installation Guide for Microsoft Windows,1. Introduction \n 1.1. System Requirements \n...,CUDA Installation Guide for Microsoft Windows\...


In [4]:
texts = df.complete_texts.to_list()

len(texts)

4856

In [6]:
print(df.iloc[0].complete_texts[:500])

CUDA Toolkit Documentation 12.5
NvidiaSearchInput.mount({"apiUrl": "https://api-prod.nvidia.com/search/graphql", "destination": "search.html", "path": "/cuda/", "site": "https://docs.nvidia.com"});
 
         
               
 Release Notes 
 CUDA Features Archive 
 EULA 
 
 Installation Guides 
 
 Quick Start Guide 
 Installation Guide Windows 
 Installation Guide Linux 
 
 Programming Guides 
 
 Programming Guide 
 Best Practices Guide 
 Maxwell Compatibility Guide 
 Pascal 


In [7]:
import re
def clean_string(s):
    cleaned_lines = [' '.join(line.split()) for line in s.splitlines()]
    cleaned_string = '\n'.join(cleaned_lines)
    cleaned_string = re.sub(r'\n+', '\n', cleaned_string)
    return cleaned_string

df["complete_texts"] = df["complete_texts"].apply(clean_string)


In [8]:
print(df.iloc[0].complete_texts[:500])

CUDA Toolkit Documentation 12.5
NvidiaSearchInput.mount({"apiUrl": "https://api-prod.nvidia.com/search/graphql", "destination": "search.html", "path": "/cuda/", "site": "https://docs.nvidia.com"});
Release Notes
CUDA Features Archive
EULA
Installation Guides
Quick Start Guide
Installation Guide Windows
Installation Guide Linux
Programming Guides
Programming Guide
Best Practices Guide
Maxwell Compatibility Guide
Pascal Compatibility Guide
Volta Compatibility Guide
Turing Compatibility Guide
NVIDI


In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
import json

hf_embeddings = HuggingFaceEmbeddings()
text_splitter = SemanticChunker(hf_embeddings)

with open("chunked_text.jsonl", "a") as f:
    for i in range(len(df)):
        text = df.iloc[i].complete_texts
        url = df.iloc[i].url
        title = df.iloc[i].title
        
        chunks = text_splitter.split_text(text)
        
        for chunk in chunks:
            f.write(json.dumps({
                "chunk": chunk,
                "url": url,
                "title": title,
            })+"\n")

In [ ]:
import pandas as pd

chunked_text = pd.read_json("chunked_text.jsonl", lines=True)

def hf_embed(chunk):
    return hf_embeddings.embed_documents([chunk])[0]

import json
from tqdm import tqdm

with open("embedded_chunked_texts.jsonl", "a") as f:
    for i, row in tqdm(df.iterrows(), total=len(df), unit="row"):
        chunk = row['chunk']
        url = row['url']
        title = row['title']
        vector = hf_embed(chunk)
        
        f.write(json.dumps({
            "chunk": chunk,
            "url": url,
            "title": title,
            "vector": vector
        }) + "\n")

In [17]:
import pandas as pd

embedded_chunks = pd.read_json("embedded_chunked_texts.jsonl", lines=True)

embedded_chunks.head()

,chunk,url,title,vector
0,CUDA Toolkit Documentation 12.5\nNvidiaSearchI...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.030554667115211, -0.047709718346595, -0.02..."
1,CUB The user guide for CUB. CUDA C++ Standard ...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.03192675486207, -0.058551348745822004, -0...."
2,cuSPARSE The cuSPARSE library user guide. NPP ...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.011653744615614001, -0.008399281650781, -0..."
3,nvFatbin The user guide for the nvFatbin libra...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.022603930905461003, -0.041946601122617, -0..."
4,Nsight Visual Studio Edition The documentation...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.015180315822362001, -0.038234677165746, -0..."


In [18]:
len(max(embedded_chunks.vector.to_list(), key=len))

768

In [19]:
len(max(embedded_chunks.chunk.to_list(), key=len))

532055

In [20]:
embedded_chunks["chunk_len"] = embedded_chunks["chunk"].apply(len)

embedded_chunks["chunk_len"].describe(percentiles=[0.25, 0.5, 0.75, 0.85, 0.9, 0.95])

count     90167.000000
mean       2096.625506
std        4142.869335
min           1.000000
25%         233.000000
50%         870.000000
75%        2593.000000
85%        3949.000000
90%        5301.000000
95%        7784.700000
max      532055.000000
Name: chunk_len, dtype: float64

In [21]:
embedded_chunks = embedded_chunks[embedded_chunks["chunk_len"]<5000]
embedded_chunks["chunk_len"].describe(percentiles=[0.25, 0.5, 0.75, 0.85, 0.9, 0.95])

count    80190.000000
mean      1174.499451
std       1233.985462
min          1.000000
25%        204.000000
50%        680.000000
75%       1826.000000
85%       2668.000000
90%       3180.000000
95%       3854.550000
max       4999.000000
Name: chunk_len, dtype: float64

In [22]:
embedded_chunks["id"] = range(len(embedded_chunks))

In [23]:
embedded_chunks.head()

,chunk,url,title,vector,chunk_len,id
1,CUB The user guide for CUB. CUDA C++ Standard ...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.03192675486207, -0.058551348745822004, -0....",496,0
2,cuSPARSE The cuSPARSE library user guide. NPP ...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.011653744615614001, -0.008399281650781, -0...",514,1
3,nvFatbin The user guide for the nvFatbin libra...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.022603930905461003, -0.041946601122617, -0...",3071,2
4,Nsight Visual Studio Edition The documentation...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.015180315822362001, -0.038234677165746, -0...",1376,3
5,Compiler SDK \nlibNVVM API The libNVVM API. l...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.024751845747232003, -0.029467459768056002,...",633,4


In [25]:
del embedded_chunks["chunk_len"]

embedded_chunks.head()

,chunk,url,title,vector,id
1,CUB The user guide for CUB. CUDA C++ Standard ...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.03192675486207, -0.058551348745822004, -0....",0
2,cuSPARSE The cuSPARSE library user guide. NPP ...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.011653744615614001, -0.008399281650781, -0...",1
3,nvFatbin The user guide for the nvFatbin libra...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.022603930905461003, -0.041946601122617, -0...",2
4,Nsight Visual Studio Edition The documentation...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.015180315822362001, -0.038234677165746, -0...",3
5,Compiler SDK \nlibNVVM API The libNVVM API. l...,https://docs.nvidia.com/cuda/,CUDA Toolkit Documentation 12.5,"[-0.024751845747232003, -0.029467459768056002,...",4


In [40]:
embedded_chunks["title"] = embedded_chunks["title"].astype(str)
embedded_chunks["url"] = embedded_chunks["url"].astype(str)
embedded_chunks["chunk"] = embedded_chunks["chunk"].astype(str)

In [41]:
from pymilvus import MilvusClient, DataType
import os

CLUSTER_ENDPOINT = "https://in03-6f08bdb85fec6ea.api.gcp-us-west1.zillizcloud.com"
TOKEN = os.getenv('ZILLIZ_TOKEN')

# 1. Set up a Milvus client
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN 
)

In [42]:
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=768)
schema.add_field(field_name="chunk", datatype=DataType.VARCHAR, max_length=8000)
schema.add_field(field_name="url", datatype=DataType.VARCHAR, max_length=1000)
schema.add_field(field_name="title", datatype=DataType.VARCHAR, max_length=1000)



client.create_collection(
    collection_name="cuda_data", 
    schema=schema, 
)

index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="HNSW",
    index_name="vector_index",
    params={ "nlist": 128 }
)

client.create_index(
    collection_name="cuda_data",
    index_params=index_params
)

In [43]:
res = client.list_indexes(
    collection_name="cuda_data"
)

print(res)

['vector_index']


In [44]:
res = client.describe_index(
    collection_name="cuda_data",
    index_name="vector_index"
)

res

{'nlist': '128',
 'index_type': 'HNSW',
 'metric_type': 'COSINE',
 'field_name': 'vector',
 'index_name': 'vector_index'}

In [45]:
data = embedded_chunks.to_json(orient="records")

In [46]:
import json

data = json.loads(data)

In [47]:
len(data)

80190

In [48]:
data_parts = [data[i * len(data) // 10: (i + 1) * len(data) // 10] for i in range(10)]

In [49]:
[len(i) for i in data_parts]

[8019, 8019, 8019, 8019, 8019, 8019, 8019, 8019, 8019, 8019]

In [51]:
for n,i in enumerate(data_parts):
    res = client.insert(
        collection_name="cuda_data",
        data=i
    )
    print(n)

0
1
2
3
4
5
6
7
8
9


In [52]:
from langchain_community.embeddings import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings()
question = "What is the CUDA version suppoorted by NVIDIA RTX"
embedded_query = hf_embeddings.embed_query(question)

In [53]:
embedded_query[:10]

[-0.007597993593662977,
 -0.0635538399219513,
 -0.024716200307011604,
 0.04655836150050163,
 -0.053499311208724976,
 -0.010638721287250519,
 0.0255771242082119,
 0.03909706696867943,
 -0.02049882523715496,
 -0.0034186614211648703]

In [98]:
client.load_collection(
    collection_name="cuda_data",
)

res = client.search(
    collection_name="cuda_data", 
    data=[embedded_query],
    limit=10, 
    search_params={"metric_type": "COSINE", "params": {}}
)

In [59]:
ids = [i["id"] for i in res[0]]
ids

[70922, 71366, 70584, 26023, 58052, 26620, 72425, 39868, 73, 25906]

In [81]:
res = client.get(
    collection_name="cuda_data",
    ids=ids
)

In [82]:
res_chunks = [[i["url"], i["chunk"]] for i in res]

res_chunks

[['https://docs.nvidia.com/cuda/cuda-features-archive/index.html',
  'CUDA Features Archive\n1. CUDA 11.6 Features\n1.1. Compiler\n1.1.1. VS2022 Support\n1.1.2. New instructions in public PTX\n1.1.3. Unused Kernel Optimization\n1.1.4. New -arch=native option\n1.1.5. Generate PTX from nvlink:\n1.1.6. Bullseye support\n1.1.7. INT128 developer tool support\n2. Notices\n2.1.'],
 ['https://docs.nvidia.com/vgpu/10.0/grid-vgpu-release-notes-vmware-vsphere/index.html',
  "x versions\nSince 10.2: 7.0, 6.7, 6.5 10.0, 10.1 only: 6.7, 6.5\nSince 10.2: 7.0, 6.7, 6.5 10.0, 10.1 only: 6.7, 6.5\nUbuntu 18.04 LTS\nSince 10.2: 7.0, 6.7, 6.5 10.0, 10.1 only: 6.7, 6.5\nSince 10.2: 7.0, 6.7, 6.5 10.0, 10.1 only: 6.7, 6.5\nUbuntu 16.04 LTS\nSince 10.2: 7.0, 6.7, 6.5 10.0, 10.1 only: 6.7, 6.5\nSince 10.2: 7.0, 6.7, 6.5 10.0, 10.1 only: 6.7, 6.5\nUbuntu 14.04 LTS\nSince 10.2: 7.0, 6.7, 6.5 10.0, 10.1 only: 6.7, 6.5\nSince 10.2: 7.0, 6.7, 6.5 10.0, 10.1 only: 6.7, 6.5\nSUSE Linux Enterprise Server 12 SP3\nSinc

In [83]:
import requests
import os
API_URL = "https://api-inference.huggingface.co/models/BAAI/bge-m3"
headers = {"Authorization": "Bearer "+str(os.getenv('HUGGINGFACE_TOKEN'))}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": {
	"source_sentence": question,
	"sentences":list([i[1] for i in res_chunks])
},
})

In [84]:
output

[0.5938552618026733,
 0.6311699151992798,
 0.6791618466377258,
 0.6869580149650574,
 0.5938552618026733,
 0.6657265424728394,
 0.6852164268493652,
 0.6842777729034424,
 0.6716938018798828,
 0.6732029914855957]

In [86]:
print("Question: "+question)
print("Context: "+" - Source. \n".join(res_chunks[output.index(max(output))]) if len(output)==len(res_chunks) else "The retreival system is still loading, Please try after 2 minutes.")

Question: What is the CUDA version suppoorted by NVIDIA RTX
Context: https://docs.nvidia.com/vgpu/9.0/grid-vgpu-release-notes-red-hat-el-kvm/index.html - Source. 
NVIDIA CUDA Toolkit Version Support
The releases in this release family of NVIDIA vGPU software support NVIDIA CUDA Toolkit 10.1 Update 1. For more information about NVIDIA CUDA Toolkit, see CUDA Toolkit 10.1 Documentation . Note:
If you are using NVIDIA vGPU software with CUDA on Linux, avoid conflicting installation methods by installing CUDA from a distribution-independent runfile package. Do not install CUDA from distribution-specific RPM or Deb package. To ensure that the NVIDIA vGPU software graphics driver is not overwritten when CUDA is installed, deselect the CUDA driver when selecting the CUDA components to install. For more information, see NVIDIA CUDA Installation Guide for Linux . 2.5. Multiple vGPU Support
To support applications and workloads that are compute or graphics intensive, multiple vGPUs can be added t

In [87]:
context = "\nSource: ".join(res_chunks[output.index(max(output))])

In [88]:
from huggingface_hub import InferenceClient
import json


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
    token=str(os.getenv('HUGGINGFACE_TOKEN'))
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 300},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [92]:
prompt = """
Your task is to answer a question given a context.
Question asked: """+ question+"""
Context provided to you from NVIDIA Documentation site: """+ context[:500]+"""
You always provide the URL source for context right after the answer.
Your answer is:
"""

In [93]:
res = call_llm(llm_client,prompt)

In [97]:
" ".join(res.split("\n")[-2:])

'The NVIDIA RTX supports CUDA Toolkit version 10.1 Update 1. Source: https://docs.nvidia.com/vgpu/9.0/grid-vgpu-release-notes-red-hat-el-kvm/index.html#cuda-toolkit-version-support'

In [1]:
## Complete functionality

from langchain_community.embeddings import HuggingFaceEmbeddings
from pymilvus import MilvusClient, DataType
import os
import requests
from huggingface_hub import InferenceClient
import json

hf_embeddings = HuggingFaceEmbeddings()


CLUSTER_ENDPOINT = "https://in03-6f08bdb85fec6ea.api.gcp-us-west1.zillizcloud.com"
TOKEN = os.getenv('ZILLIZ_TOKEN')

client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN 
)

API_URL = "https://api-inference.huggingface.co/models/BAAI/bge-m3"
headers = {"Authorization": "Bearer "+str(os.getenv('HUGGINGFACE_TOKEN'))}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": {
	"source_sentence": ["placeholder_to_load_the_model"],
	"sentences":["placeholder_to_load_the_model", "placeholder_to_load_the_model"],
},
})


llm_client = InferenceClient(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    timeout=120,
    token=str(os.getenv('HUGGINGFACE_TOKEN'))
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

def chatbot_chat(question):
    embedded_query = hf_embeddings.embed_query(question)

    client.load_collection(
    collection_name="cuda_data",
    )

    res = client.search(
        collection_name="cuda_data", 
        data=[embedded_query],
        limit=10, 
        search_params={"metric_type": "COSINE", "params": {}}
    )
    ids = [i["id"] for i in res[0]]
    res = client.get(
        collection_name="cuda_data",
        ids=ids
    )

    res_chunks = [[i["url"], i["chunk"]] for i in res]

    output = query({
        "inputs": {
        "source_sentence": question,
        "sentences":list([i[1] for i in res_chunks])
    },
    })

    context = "\nSource: ".join(res_chunks[output.index(max(output))])
    
    prompt = """
Your task is to answer a question given a context.
Question asked: """+ question+"""
Context provided to you from NVIDIA Documentation site: """+ context[:1500]+"""
You always provide the URL source for context right after the answer.
Your answer is:
"""
    res = call_llm(llm_client,prompt)
    return " ".join(res.split("\n")[-2:])


/home/bharathraj/anaconda3/envs/nemo_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [2]:
chatbot_chat("What are the new features of CUDA 11.6?")

'Your answer is: The new features of CUDA 11.6 include support for VS2022 as a host compiler, new instructions for bit mask creation (BMSK) and sign extension (SZEXT) in public PTX, and unused kernel optimization enabled by default.'

In [3]:
chatbot_chat("What is NVIDIA NPP?")

'NVIDIA Performance Primitives (NPP) is a library of functions for performing image and signal processing operations. It is designed to provide high performance on NVIDIA GPUs. NPP provides a wide range of functions for image filtering, color space conversion, morphological operations, and more. It is intended to be used as a building block for higher-level image processing and computer vision applications. Source: https://docs.nvidia.com/cuda/npp/introduction.html'